# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
experiment = 'DDB_PD_060_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Comments,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Experiment Annotation,growth_rate,...,Temperature Setpoint (C),Feed #1 Profile,pH Setpoint,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm)
0,DDB_PD_060_AMBR_DDB35-001_R06_Main_,DDB_BM_PD60_R06,Variations in media prep are documented in Med...,R06,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000
1,DDB_PD_060_AMBR_DDB35-001_R07_Main_,DDB_BM_PD60_R07,Variations in media prep are documented in Med...,R07,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000
2,DDB_PD_060_AMBR_DDB35-001_R08_Main_,DDB_BM_PD60_R08,Variations in media prep are documented in Med...,R08,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000
3,DDB_PD_060_AMBR_DDB35-001_R09_Main_,DDB_BM_PD60_R09,Variations in media prep are documented in Med...,R09,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000
4,DDB_PD_060_AMBR_DDB35-001_R10_Main_,DDB_BM_PD60_R10,Variations in media prep are documented in Med...,R10,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000
5,DDB_PD_060_AMBR_DDB35-001_R11_Main_,DDB_BM_PD60_R11,Variations in media prep are documented in Med...,R11,AMBR 250,No,Main,DDB_PD_060_AMBR,DDB Trace Metals,0.11,...,30,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,Feed #1 Profile,pH Setpoint,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Phase
0,0.000000,95.432912,6.494567,999.597570,29.870001,4.091890,99.997146,0,20.95,20.716432,...,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000,batch
1,0.033333,93.725190,6.493214,998.928584,29.880713,4.113640,99.940289,0,20.95,20.716713,...,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000,batch
2,0.066667,92.654450,6.494210,998.209833,29.889999,3.957790,100.026036,0,20.95,20.709104,...,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000,batch
3,0.100000,91.761919,6.497431,999.142204,29.918578,1.929151,100.057221,0,20.95,20.713284,...,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000,batch
4,0.133333,91.449055,6.495424,1000.788142,29.940001,4.068780,100.075082,0,20.95,20.705907,...,DDB_Feed_Profile_DDB-II (only exponential feed),6.5,Ammonium hydroxide 15%,40,Stirring,0.1,0.1,1000,4000,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_060_AMBR_R06__S01,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S01,0.000000,2,0.3500
1,DDB_PD_060_AMBR_R06__S02,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S02,4.019164,2,1.3710
2,DDB_PD_060_AMBR_R06__S03,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S03,8.018451,2,5.5050
3,DDB_PD_060_AMBR_R06__S04,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S04,10.016866,2,12.4900
4,DDB_PD_060_AMBR_R06__S05,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S05,12.020319,2,22.2377


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_060_AMBR_R06__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,2021-08-23,0.0,DDB_PD_060_AMBR_R06__S01,succinic acid,0.00000,g/L
1,DDB_PD_060_AMBR_R06__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,2021-08-23,0.0,DDB_PD_060_AMBR_R06__S01,ethanol,0.00000,g/L
2,DDB_PD_060_AMBR_R06__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,2021-08-23,0.0,DDB_PD_060_AMBR_R06__S01,citric acid,0.00000,g/L
3,DDB_PD_060_AMBR_R06__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,2021-08-23,0.0,DDB_PD_060_AMBR_R06__S01,D-glucose,27.20664,g/L
4,DDB_PD_060_AMBR_R06__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,2021-08-23,0.0,DDB_PD_060_AMBR_R06__S01,formic acid,0.00000,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,27.206640,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,DDB_PD_060_AMBR_R06__S01
1,23.771038,0.435215,0.0,0.000000,0.0,0.000000,0.057680,0.000000,DDB_PD_060_AMBR_R06__S02
2,18.066339,1.137429,0.0,0.000000,0.0,1.837420,0.775019,0.183202,DDB_PD_060_AMBR_R06__S03
3,13.091076,1.819179,0.0,0.000000,0.0,2.940084,0.296910,0.298295,DDB_PD_060_AMBR_R06__S04
4,0.000000,1.946040,0.0,2.080206,0.0,1.753572,0.184303,0.255411,DDB_PD_060_AMBR_R06__S05


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample
0,DDB_PD_060_AMBR_R06__S06_TM_#1,mn,97.476669,µg/L,DDB_PD_060_AMBR_R06__S06
1,DDB_PD_060_AMBR_R06__S06_TM_#1,fe,3615.958483,µg/L,DDB_PD_060_AMBR_R06__S06
2,DDB_PD_060_AMBR_R06__S06_TM_#1,co,129.921386,µg/L,DDB_PD_060_AMBR_R06__S06
3,DDB_PD_060_AMBR_R06__S06_TM_#1,cu,38.807099,µg/L,DDB_PD_060_AMBR_R06__S06
4,DDB_PD_060_AMBR_R06__S06_TM_#1,zn,0.000000,µg/L,DDB_PD_060_AMBR_R06__S06


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,co [µg/L],cu [µg/L],fe [µg/L],k [mg/L],mg [mg/L],mn [µg/L],mo [µg/L],na [mg/L],s [mg/L],zn [µg/L],Timepoint Sample
0,132.048858,138.710128,4700.211306,5852.326874,19.705926,67.889122,143.058004,122.677913,2714.388509,163.117700,DDB_PD_060_AMBR_R06__S01
1,138.161488,171.890293,4879.012584,5746.966289,29.163660,83.837115,150.264386,159.244769,2348.622062,162.808341,DDB_PD_060_AMBR_R06__S02
2,142.616178,153.204807,5060.496019,5302.328149,48.172508,22.946659,142.201107,74.191926,2365.503541,163.081811,DDB_PD_060_AMBR_R06__S03
3,154.210668,127.740127,5078.741176,5865.325977,44.745117,30.593394,147.235678,80.964898,2624.005566,84.028297,DDB_PD_060_AMBR_R06__S04
4,143.063181,88.099430,4155.152730,5231.134948,36.657618,84.635861,121.411879,73.617681,2384.933708,0.000000,DDB_PD_060_AMBR_R06__S05


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],...,co [µg/L],cu [µg/L],fe [µg/L],k [mg/L],mg [mg/L],mn [µg/L],mo [µg/L],na [mg/L],s [mg/L],zn [µg/L]
0,DDB_PD_060_AMBR_R06__S01,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S01,0.000000,2,0.3500,27.206640,0.000000,...,132.048858,138.710128,4700.211306,5852.326874,19.705926,67.889122,143.058004,122.677913,2714.388509,163.117700
1,DDB_PD_060_AMBR_R06__S02,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S02,4.019164,2,1.3710,23.771038,0.435215,...,138.161488,171.890293,4879.012584,5746.966289,29.163660,83.837115,150.264386,159.244769,2348.622062,162.808341
2,DDB_PD_060_AMBR_R06__S03,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S03,8.018451,2,5.5050,18.066339,1.137429,...,142.616178,153.204807,5060.496019,5302.328149,48.172508,22.946659,142.201107,74.191926,2365.503541,163.081811
3,DDB_PD_060_AMBR_R06__S04,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S04,10.016866,2,12.4900,13.091076,1.819179,...,154.210668,127.740127,5078.741176,5865.325977,44.745117,30.593394,147.235678,80.964898,2624.005566,84.028297
4,DDB_PD_060_AMBR_R06__S05,DDB_PD_060_AMBR,DDB_PD_060_AMBR_DDB35-001_R06_Main_,R06,S05,12.020319,2,22.2377,0.000000,1.946040,...,143.063181,88.099430,4155.152730,5231.134948,36.657618,84.635861,121.411879,73.617681,2384.933708,0.000000


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)